In [20]:
import sys
sys.path.append('/home/thiago/Documentos/GitHub/pipeline_de_dados/utils')
from spark_config import SparkConfig
from pyspark.sql import functions as F

In [21]:
spark = SparkConfig()

In [22]:
spark= SparkConfig().spark_config()

In [23]:
df = spark.read.format('delta').load('s3a://datalake-test-thiago/03-gold/delta/store_sales')

In [24]:
num_linhas = df.count()
print(f"Número de linhas no DataFrame: {num_linhas}")

Número de linhas no DataFrame: 1017209


In [25]:
num_colunas = len(df.columns)
print(f"Número de colunas no DataFrame: {num_colunas}")

Número de colunas no DataFrame: 29


In [26]:
df = spark.read.format('parquet').load('s3a://datalake-test-thiago/01-bronze/spark/train')


In [27]:
num_linhas = df.count()
print(f"Número de linhas no DataFrame: {num_linhas}")

Número de linhas no DataFrame: 1017209


In [28]:
df = spark.read.format('delta').load('s3a://datalake-test-thiago/02-silver/delta/store_sales')


In [29]:
num_linhas = df.count()
print(f"Número de linhas no DataFrame: {num_linhas}")

Número de linhas no DataFrame: 1017209


In [30]:
df = spark.read.format('delta').load('s3a://datalake-test-thiago/03-gold/delta/store_sales')


In [31]:
num_linhas = df.count()
print(f"Número de linhas no DataFrame: {num_linhas}")

Número de linhas no DataFrame: 1017209
